### Import libraries and constants

In [76]:
import os
import pandas as pd

PATH_EXCEL = os.path.abspath("../data/Datos Maestros VF.xlsx")
PATH_TXT = os.path.abspath("../data/dDEC1204.TXT")
PATH_TO_SAVE = os.path.abspath("./Dataset.csv") 

### Load Excel into pd.DataFrame

In [ ]:
df_excel_raw = pd.read_excel(open(PATH_EXCEL, 'rb'),
              sheet_name='Master Data Oficial')

In [ ]:
#df_excel_raw.columns

dic_columns = { 'Nombre visible Agente':'AGENTE_VISIBLE'
                ,'AGENTE (OFEI)':'AGENTE_OFEI'
                ,'CENTRAL (dDEC, dSEGDES, dPRU…)':'CENTRAL'
                ,'Tipo de central (Hidro, Termo, Filo, Menor)':'TIPO_CENTRAL'}

for name, rename in dic_columns.items():
    df_excel_raw.rename(columns={name: rename}, inplace=True)


In [ ]:
select_columns_excel = list(dic_columns.values())
df_excel_selected = df_excel_raw[select_columns_excel].copy()


In [36]:
df_excel_filtered = df_excel_selected[((df_excel_selected['AGENTE_VISIBLE'] == "EMGESA")
                                        | (df_excel_selected['AGENTE_OFEI'] == "EMGESA S.A."))
                                        & (df_excel_selected['TIPO_CENTRAL'].isin(['H', 'T']))]


In [82]:
df_excel_filtered.head(5)

,AGENTE_VISIBLE,AGENTE_OFEI,CENTRAL,TIPO_CENTRAL
32,EMGESA,EMGESA S.A.,BETANIA,H
33,EMGESA,EMGESA S.A.,BETANIA,H
34,EMGESA,EMGESA S.A.,BETANIA,H
49,EMGESA,EMGESA S.A.,CTGEMG1,T
50,EMGESA,EMGESA S.A.,CTGEMG2,T


### Load .text to pd.DataFrame

In [61]:
columns = ["CENTRAL"] + ["Hora_{}".format(i) for i in range(1, 25)]
df_text = pd.read_csv(PATH_TXT, encoding="latin1")
df_text.columns = columns

In [83]:
df_text.head(5)

,CENTRAL,Hora_1,Hora_2,Hora_3,Hora_4,Hora_5,Hora_6,Hora_7,Hora_8,Hora_9,...,Hora_15,Hora_16,Hora_17,Hora_18,Hora_19,Hora_20,Hora_21,Hora_22,Hora_23,Hora_24
0,AG_ARGOSYUM,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AG_ELCAIRO,0,0,0,1,3,3,3,3,3,...,3,0,0,0,0,1,3,3,3,3
2,AG_REFICAR,9,9,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
3,AG_UNIBOL,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,AG_YAGUARI,0,1,1,1,0,1,0,1,1,...,0,1,1,1,1,1,1,1,1,0


### Merging Dataframes by central

In [64]:
df_merged = pd.merge(df_excel_filtered, df_text, on="CENTRAL", how="left")

In [84]:
df_merged.head(5)

,AGENTE_VISIBLE,AGENTE_OFEI,CENTRAL,TIPO_CENTRAL,Hora_1,Hora_2,Hora_3,Hora_4,Hora_5,Hora_6,...,Hora_16,Hora_17,Hora_18,Hora_19,Hora_20,Hora_21,Hora_22,Hora_23,Hora_24,SUM_OF_HOURS
0,EMGESA,EMGESA S.A.,BETANIA,H,364,364,364,364,364,364,...,364,364,364,364,364,364,364,364,364,8736
1,EMGESA,EMGESA S.A.,BETANIA,H,364,364,364,364,364,364,...,364,364,364,364,364,364,364,364,364,8736
2,EMGESA,EMGESA S.A.,BETANIA,H,364,364,364,364,364,364,...,364,364,364,364,364,364,364,364,364,8736
3,EMGESA,EMGESA S.A.,CTGEMG1,T,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,EMGESA,EMGESA S.A.,CTGEMG2,T,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Sum the hours horizontally

In [72]:
columns_to_sum = ["Hora_{}".format(i) for i in range(1, 25)]
df_merged["SUM_OF_HOURS"]=df_merged[columns_to_sum].sum(axis=1)

In [85]:
df_merged.head(5)

,AGENTE_VISIBLE,AGENTE_OFEI,CENTRAL,TIPO_CENTRAL,Hora_1,Hora_2,Hora_3,Hora_4,Hora_5,Hora_6,...,Hora_16,Hora_17,Hora_18,Hora_19,Hora_20,Hora_21,Hora_22,Hora_23,Hora_24,SUM_OF_HOURS
0,EMGESA,EMGESA S.A.,BETANIA,H,364,364,364,364,364,364,...,364,364,364,364,364,364,364,364,364,8736
1,EMGESA,EMGESA S.A.,BETANIA,H,364,364,364,364,364,364,...,364,364,364,364,364,364,364,364,364,8736
2,EMGESA,EMGESA S.A.,BETANIA,H,364,364,364,364,364,364,...,364,364,364,364,364,364,364,364,364,8736
3,EMGESA,EMGESA S.A.,CTGEMG1,T,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,EMGESA,EMGESA S.A.,CTGEMG2,T,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Select the restulting sums grather than 0

In [75]:
df_final = df_merged[df_merged["SUM_OF_HOURS"] > 0]

In [86]:
df_final.head(5)

,AGENTE_VISIBLE,AGENTE_OFEI,CENTRAL,TIPO_CENTRAL,Hora_1,Hora_2,Hora_3,Hora_4,Hora_5,Hora_6,...,Hora_16,Hora_17,Hora_18,Hora_19,Hora_20,Hora_21,Hora_22,Hora_23,Hora_24,SUM_OF_HOURS
0,EMGESA,EMGESA S.A.,BETANIA,H,364,364,364,364,364,364,...,364,364,364,364,364,364,364,364,364,8736
1,EMGESA,EMGESA S.A.,BETANIA,H,364,364,364,364,364,364,...,364,364,364,364,364,364,364,364,364,8736
2,EMGESA,EMGESA S.A.,BETANIA,H,364,364,364,364,364,364,...,364,364,364,364,364,364,364,364,364,8736
6,EMGESA,EMGESA S.A.,ELQUIMBO,H,85,85,85,85,85,85,...,85,85,85,85,85,85,85,85,85,2040
7,EMGESA,EMGESA S.A.,ELQUIMBO,H,85,85,85,85,85,85,...,85,85,85,85,85,85,85,85,85,2040


### Save the resulting dataframe

In [77]:
df_final.to_csv(PATH_TO_SAVE)